# Limpieza de Datos y Organización

### Limpiamos y unimos los datos de progesores con su enlace a las reviews

In [13]:
import pandas as pd
import os
from pathlib import Path
import re
import unicodedata

In [2]:
# Parent file path
DIR = os.getcwd()
print(f"Current working directory: {DIR}")
parent_file = Path(DIR).parent
print(f"Parent file path: {parent_file}")
# Change dir
os.chdir(parent_file)

Current working directory: c:\Users\crish\Desktop\Profesor_Resume\scrap
Parent file path: c:\Users\crish\Desktop\Profesor_Resume


In [3]:
data_prof = pd.read_csv('data/tabla_profesores.csv')
enlaces_df = pd.read_csv('data/enlaces_profesores.csv')

In [4]:
data_prof.head(5)

,profesor,dep,num_reviews,rating
0,"- AMLO 2018, nuestro precidente",chingonerias,78,6.2
1,"..., Paola Eunice",psicología social,8,10.0
2,"45675094, Rodolfo Márquez 4 567 5094",Rm0872926@gmail.com,1,10.0
3,"A. Mason, Tere",Psicología/CCC,3,6.7
4,"Abrego González, Leticia",Psicología Clínica,14,3.6


In [5]:
data_prof.drop(axis = 0, index = range(0, 3), inplace = True)

In [6]:
data_prof.head()

,profesor,dep,num_reviews,rating
3,"A. Mason, Tere",Psicología/CCC,3,6.7
4,"Abrego González, Leticia",Psicología Clínica,14,3.6
5,"Acosta Galván, Samuel",clínica,18,7.3
6,"Acuña, Laura",Estadistica,27,6.2
7,"Aguayo, Alejandra",NaN,13,7.2


In [32]:
# Definimos nombre sin acentos
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii.decode('utf-8')

# Aplicamos la función a la columna 'profesor'
data_prof['name'] = data_prof['profesor'].apply(remove_accents).str.lower()

In [49]:
data_prof.head(15)

,profesor,dep,num_reviews,rating,name
3,"A. Mason, Tere",Psicología/CCC,3,6.7,"a. mason, tere"
4,"Abrego González, Leticia",Psicología Clínica,14,3.6,"abrego gonzalez, leticia"
5,"Acosta Galván, Samuel",clínica,18,7.3,"acosta galvan, samuel"
6,"Acuña, Laura",Estadistica,27,6.2,"acuna, laura"
7,"Aguayo, Alejandra",NaN,13,7.2,"aguayo, alejandra"
8,"Aguilar Arredondo, Andrea",psicología,1,9.0,"aguilar arredondo, andrea"
9,"Aguilar Nery, Jesús",Psicología de la Educación,1,9.0,"aguilar nery, jesus"
10,"Aguilar Ribera, UNIBERCIDAD.AUTÓNOMA DE MEXICO",Piciocologuia,1,10.0,"aguilar ribera, unibercidad.autonoma de mexico"
11,"ALANIS, MARIA TERESA",NaN,9,6.7,"alanis, maria teresa"
12,"alatorre rico, javier",psicologia/ educativa,89,5.4,"alatorre rico, javier"


In [7]:
enlaces_df = enlaces_df.drop(index = range(6), axis=0)

In [8]:
enlaces_df.head(10)

,enlace
6,https://www.misprofesores.com/profesores/Tere-...
7,https://www.misprofesores.com/profesores/Letic...
8,https://www.misprofesores.com/profesores/Samue...
9,https://www.misprofesores.com/profesores/Beren...
10,https://www.misprofesores.com/profesores/Laura...
11,https://www.misprofesores.com/profesores/Aleja...
12,https://www.misprofesores.com/profesores/Andre...
13,https://www.misprofesores.com/profesores/Jesus...
14,https://www.misprofesores.com/profesores/UNIBE...
15,https://www.misprofesores.com/profesores/John-...


In [41]:
regex = r"(https://www\.misprofesores\.com/profesores/)"

def extract_professor_name(url: str) -> str:
    """
    Extracts and formats a professor's name from the URL.
    Args:
        url (str): The URL containing the professor's name.
    Returns:
        str: The formatted professor's name as 'Apellidos, Nombres'
    """
    match = re.search(regex, url)
    if not match:
        return ""

    # Elimina el prefijo de URL y el número final
    name_number = url.replace(match.group(0), "").split('_')[0]

    # Divide por guiones
    parts = name_number.split('-')

    # Si hay solo 2 partes, asumimos nombre + apellido
    if len(parts) == 2:
        nombre = parts[0]
        apellidos = parts[1]
    # Si hay 3 partes, asumimos nombre + apellido1 + apellido2
    elif len(parts) == 3:
        nombre = parts[0]
        apellidos = " ".join(parts[1:])
    # Si hay 4 o más, asumimos últimos 2 son apellidos, el resto nombre(s)
    elif len(parts) >= 4:
        nombre = " ".join(parts[:-2])
        apellidos = " ".join(parts[-2:])
    else:
        return name_number.lower()

    return f"{apellidos}, {nombre}".lower()

enlaces_df["name"] = enlaces_df['enlace'].apply(extract_professor_name)

In [42]:
enlaces_df.head(15)

,enlace,name
6,https://www.misprofesores.com/profesores/Tere-...,"a. mason, tere"
7,https://www.misprofesores.com/profesores/Letic...,"abrego gonzalez, leticia"
8,https://www.misprofesores.com/profesores/Samue...,"acosta galvan, samuel"
9,https://www.misprofesores.com/profesores/Beren...,"acosta, berenice"
10,https://www.misprofesores.com/profesores/Laura...,"acuna, laura"
11,https://www.misprofesores.com/profesores/Aleja...,"aguayo, alejandra"
12,https://www.misprofesores.com/profesores/Andre...,"aguilar arredondo, andrea"
13,https://www.misprofesores.com/profesores/Jesus...,"aguilar nery, jesus"
14,https://www.misprofesores.com/profesores/UNIBE...,"aguilar ribera, unibercidad.autonoma de mexico"
15,https://www.misprofesores.com/profesores/John-...,"akerman, john"


In [45]:
# Left join the dataframes
merged_df = pd.merge(data_prof, enlaces_df, on = "name", how="left")

In [46]:
merged_df.head(20)

,profesor,dep,num_reviews,rating,name,enlace
0,"A. Mason, Tere",Psicología/CCC,3,6.7,"a. mason, tere",https://www.misprofesores.com/profesores/Tere-...
1,"Abrego González, Leticia",Psicología Clínica,14,3.6,"abrego gonzalez, leticia",https://www.misprofesores.com/profesores/Letic...
2,"Acosta Galván, Samuel",clínica,18,7.3,"acosta galvan, samuel",https://www.misprofesores.com/profesores/Samue...
3,"Acuña, Laura",Estadistica,27,6.2,"acuna, laura",https://www.misprofesores.com/profesores/Laura...
4,"Aguayo, Alejandra",NaN,13,7.2,"aguayo, alejandra",https://www.misprofesores.com/profesores/Aleja...
5,"Aguilar Arredondo, Andrea",psicología,1,9.0,"aguilar arredondo, andrea",https://www.misprofesores.com/profesores/Andre...
6,"Aguilar Nery, Jesús",Psicología de la Educación,1,9.0,"aguilar nery, jesus",https://www.misprofesores.com/profesores/Jesus...
7,"Aguilar Ribera, UNIBERCIDAD.AUTÓNOMA DE MEXICO",Piciocologuia,1,10.0,"aguilar ribera, unibercidad.autonoma de mexico",https://www.misprofesores.com/profesores/UNIBE...
8,"ALANIS, MARIA TERESA",NaN,9,6.7,"alanis, maria teresa",NaN
9,"alatorre rico, javier",psicologia/ educativa,89,5.4,"alatorre rico, javier",https://www.misprofesores.com/profesores/javie...


In [47]:
# NAs en merged_df
merged_df.isna().sum()

profesor        0
dep            59
num_reviews     0
rating          0
name            0
enlace         46
dtype: int64

In [51]:
# Vemos donde están los NAs de enlace en merged_df
na_enlaces = merged_df[merged_df['enlace'].isna()]
print(na_enlaces[['name', 'profesor', 'num_reviews']])

                                name                         profesor  \
8               alanis, maria teresa             ALANIS, MARIA TERESA   
33       augusto, martinez ruvalcaba      Augusto, Martínez Ruvalcaba   
47                 baroja, jose luis                Baroja, José Luis   
72              brito, luis fernando             Brito, luis Fernando   
105         castaneda, irma graciela         Castañeda, Irma Graciela   
111             cervantes, ana karen             Cervantes, Ana Karen   
134         conroy, maria del carmen         conroy, maria del carmen   
158              david, diaz mercado              David, Díaz Mercado   
183  dominguez, alejandra del carmen  Dominguez, Alejandra del Carmen   
193             diaz-loving, rolando             Díaz-Loving, Rolando   
195          edgar, gutierrez cuevas          Edgar, Gutiérrez Cuevas   
231            flores, ma. guadalupe            flores, ma. guadalupe   
233     freud, hector bautista ortiz     Freud, Héc

In [52]:
# Save Merged DataFrame
merged_df.to_csv('data/merged_profesores.csv', index=False)

In [53]:
# Save enlaces_df
enlaces_df.to_csv('data/enlaces_profesores_clean.csv', index=False)